# Question 2: Resampling and Frequency Conversion

This question focuses on resampling operations and frequency conversion using ICU monitoring data (hourly) and patient vital signs data (daily).

## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os

# Set random seed for reproducibility
np.random.seed(42)

# Set plotting style
plt.style.use('default')
sns.set_style('whitegrid')

# Create output directory
os.makedirs('output', exist_ok=True)

## Part 2.1: Load and Prepare Data

**Note:** These datasets have realistic characteristics:
- **ICU Monitoring**: 75 patients with variable stay lengths (2-30 days). Not all patients are present for the entire 6-month period - patients are admitted and discharged at different times.
- **Patient Vitals**: Already contains some missing visits (~5% missing data). This is realistic and will be useful for practicing missing data handling.

In [2]:
# Load ICU monitoring data (hourly)
icu_monitoring = pd.read_csv('data/icu_monitoring.csv')

# Load patient vitals data (daily) - for comparison
patient_vitals = pd.read_csv('data/patient_vitals.csv')

print("ICU monitoring shape:", icu_monitoring.shape)
print("Patient vitals shape:", patient_vitals.shape)

# Convert datetime columns and set as index
icu_monitoring['datetime'] = pd.to_datetime(icu_monitoring['datetime'])
icu_monitoring = icu_monitoring.set_index('datetime')

patient_vitals['date'] = pd.to_datetime(patient_vitals['date'])
patient_vitals = patient_vitals.set_index('date')

print("\nICU monitoring sample:")
print(icu_monitoring.head())
print("\nPatient vitals sample:")
print(patient_vitals.head())

# Check data characteristics
print(f"\nICU patients: {icu_monitoring['patient_id'].nunique()}")
print(f"ICU date range: {icu_monitoring.index.min()} to {icu_monitoring.index.max()}")
print(f"\nPatient vitals patients: {patient_vitals['patient_id'].nunique()}")
print(f"Patient vitals date range: {patient_vitals.index.min()} to {patient_vitals.index.max()}")

ICU monitoring shape: (86400, 7)
Patient vitals shape: (18250, 7)

ICU monitoring sample:
                    patient_id  heart_rate  blood_pressure_systolic  \
datetime                                                              
2023-01-01 00:00:00     ICU001   82.000000                      126   
2023-01-01 01:00:00     ICU001   98.294095                      128   
2023-01-01 02:00:00     ICU001  103.500000                      129   
2023-01-01 03:00:00     ICU001   91.535534                      136   
2023-01-01 04:00:00     ICU001   87.330127                      129   

                     blood_pressure_diastolic  oxygen_saturation  temperature  
datetime                                                                       
2023-01-01 00:00:00                        65                 96    98.783988  
2023-01-01 01:00:00                        67                 95    99.186212  
2023-01-01 02:00:00                        68                 94    98.800638  
2023-01-01 0

## Part 2.2: Time Series Selection

**⚠️ WARNING: Sort Index Before Date Selection!**
Since multiple patients share the same date, the `patient_vitals` index is non-monotonic (not strictly increasing). **You MUST sort the index first** before using `.loc` with date ranges:

In [3]:
patient_vitals = patient_vitals.sort_index()

Without sorting, pandas cannot reliably handle date range selections and may return unexpected results or errors.

**TODO: Perform time series indexing and selection**

In [9]:
# TODO: Select data by specific dates
# Note: Not all patients may have data on January 1, 2023 (some start later)
# Important: Sort the index first since multiple patients share the same date
# patient_vitals = patient_vitals.sort_index()  # Sort for reliable date-based selection
january_first = patient_vitals.loc["2023-01-23"]  # Select January 1, 2023 from patient_vitals
print("January 1, 2023 data:", january_first)
print(f"Records on Jan 1: {len(january_first)} (some patients may start later)")

# TODO: Select data by date ranges
january_data = patient_vitals.loc["2023-01-01":"2023-01-31"]
print("January 2023 shape:", january_data.shape)

# TODO: Select data by time periods
quarterly = pd.date_range('2023-01-01','2023-12-31', freq = 'QS')
print(quarterly)
first_quarter = patient_vitals.loc['2023-01-01':'2023-04-01'] #select Q1 2023
entire_year = patient_vitals.loc['2023']  # Select all of 2023 (will include patients with partial year data)

# TODO: Select first and last periods using .loc
first_week = patient_vitals.loc[:patient_vitals.index.min() + pd.Timedelta(days=6)]  # First 7 days
last_week = patient_vitals.loc[patient_vitals.index.max() - pd.Timedelta(days=6):]  # Last 7 days

# TODO: Use truncate() method
# Note: truncate() requires a sorted index. Sort first if needed: patient_vitals = patient_vitals.sort_index()
data_after_june = patient_vitals.truncate(before='2023-06-01')  # Truncate before June 1, 2023
data_before_september = patient_vitals.truncate(after='2023-08-31')  # Truncate after August 31, 2023

# TODO: Use selected data for analysis
# Compare average temperature between first quarter and data after June
print(f"\nFirst quarter average temperature: {first_quarter['temperature'].mean():.2f}°F")
print(f"After June average temperature: {data_after_june['temperature'].mean():.2f}°F")
print(f"First week average temperature: {first_week['temperature'].mean():.2f}°F")
print(f"Last week average temperature: {last_week['temperature'].mean():.2f}°F")

# For ICU data with time components:
# TODO: Select business hours (9 AM to 5 PM)
business_hours = icu_monitoring.between_time('9:00','17:00')  # Use between_time()
print("Business hours data shape:", business_hours.shape)

# TODO: Select specific time (noon readings)
noon_data = icu_monitoring.at_time('12:00')  # Use at_time('12:00')

# TODO: Use time-based selection for analysis
# Compare vital signs during business hours vs other times
all_hours_avg = icu_monitoring.select_dtypes(include=[np.number]).mean()
business_hours_avg = business_hours.select_dtypes(include=[np.number]).mean()
print(f"\nAverage heart rate - All hours: {all_hours_avg['heart_rate']:.1f} bpm")
print(f"Average heart rate - Business hours: {business_hours_avg['heart_rate']:.1f} bpm")
print(f"Average temperature - All hours: {all_hours_avg['temperature']:.1f}°F")
print(f"Average temperature - Business hours: {business_hours_avg['temperature']:.1f}°F")

January 1, 2023 data:            patient_id  temperature  heart_rate  blood_pressure_systolic  \
date                                                                      
2023-01-23      P0044    98.563784          61                      112   
2023-01-23      P0002    99.239016          86                      105   
2023-01-23      P0005    97.740030          57                      118   
2023-01-23      P0043    98.727877          85                      110   
2023-01-23      P0027    98.454538          93                      117   
2023-01-23      P0014    98.212433          78                      118   
2023-01-23      P0013    99.057357          71                      117   
2023-01-23      P0017    99.239211          79                      112   
2023-01-23      P0021    99.851922          76                      125   
2023-01-23      P0050    99.456538          74                      125   
2023-01-23      P0039    99.481110          93                      131   
202

## Part 2.3: Resampling Operations

**TODO: Perform resampling and frequency conversion**

**Important Note:** When resampling DataFrames that contain non-numeric columns (like `patient_id`), you'll get an error if you try to aggregate them with numeric functions like `mean()`. Use `df.select_dtypes(include=[np.number])` to select only numeric columns before resampling, or specify which columns to aggregate in `.agg()`.

In [16]:
# TODO: Resample hourly ICU data to daily
# Note: Exclude non-numeric columns like 'patient_id' when resampling
# Select only numeric columns before resampling
numeric_cols = icu_monitoring.select_dtypes(include=[np.number]).columns
icu_daily = icu_monitoring[numeric_cols].resample('D').mean()
print("ICU daily shape:", icu_daily.shape)

# TODO: Resample daily patient data to weekly
# Note: Exclude 'patient_id' column when resampling
# Select only numeric columns before resampling
numeric_cols_pv = patient_vitals.select_dtypes(include=[np.number]).columns
patient_vitals_weekly = patient_vitals[numeric_cols_pv].resample('W').mean()
print("Weekly resampled shape:", patient_vitals_weekly.shape)

# TODO: Resample daily patient data to monthly
patient_vitals_monthly = patient_vitals[numeric_cols_pv].resample('ME').mean()  # Resample to monthly with mean aggregation (use freq='ME' for Month End)
print("Monthly resampled shape:", patient_vitals_monthly.shape)

# TODO: Use different aggregation functions (mean, sum, max, min)
icu_daily_stats = icu_monitoring.resample('D').agg({'heart_rate': ['mean', 'max', 'min'], 
                         'temperature': 'mean'})  # Resample with multiple aggregations


# TODO: Handle missing values during resampling
# Demonstrate upsampling (monthly to daily) creates missing values
# Note: When upsampling, use .asfreq() to create missing values, or use .resample() with aggregation
monthly_to_daily = patient_vitals_monthly.asfreq('D')  # Upsample monthly data to daily (use .asfreq() or .resample('D'))
print("Missing values after upsampling:", monthly_to_daily.isna().sum())

# TODO: Compare different resampling frequencies
# Create a DataFrame comparing resampling results at different frequencies
# Important: Since patient_vitals contains multiple patients per date, you need to aggregate by date first
# to create a single daily time series for comparison.
# Why aggregation is needed: The patient_vitals DataFrame has multiple rows per date (one for each patient),
# so we need to average across patients for each date to create a single daily time series that can be
# meaningfully compared with the weekly and monthly resampled data. Without aggregation, resampling would
# operate on each patient's time series separately, making it difficult to compare frequencies meaningfully.
# Steps:
# 1. Since 'date' is currently the index, reset it to a column first, then aggregate by date
#    Note: groupby('date').mean() automatically sets 'date' as the index in the result, so you don't need
#    to call set_index('date') again after groupby.
patient_vitals_reset = patient_vitals[numeric_cols_pv].reset_index()
patient_vitals_daily_agg = patient_vitals_reset.groupby('date').mean()
#    # The date is already the index after groupby, so no need to set_index again
# 2. Compare the aggregated daily data with weekly and monthly resampled data
# Use patient_vitals data resampled to different frequencies:
# - Original daily data (aggregated by date): patient_vitals_daily_agg
# - Weekly resampled (patient_vitals_weekly) 
# - Monthly resampled (patient_vitals_monthly)
# Include columns: frequency, date_range, row_count, mean_temperature, std_temperature
# Use the 'temperature' column from each resampled dataset
# Example structure:
resampling_comparison = pd.DataFrame({
     'frequency': ['daily', 'weekly', 'monthly'],
     'date_range': [
         f"{patient_vitals_daily_agg.index.min().date()} - {patient_vitals_daily_agg.index.max().date()}",
         f"{patient_vitals_weekly.index.min().date()} - {patient_vitals_weekly.index.max().date()}",
         f"{patient_vitals_monthly.index.min().date()} - {patient_vitals_monthly.index.max().date()}"],  # Use index.min() and index.max() for each dataset
     'row_count': [
         len(patient_vitals_daily_agg),
         len(patient_vitals_weekly),
         len(patient_vitals_monthly)
     ],  # Use len() for each dataset
     'mean_temperature': [patient_vitals_daily_agg['temperature'].mean(),
        patient_vitals_weekly['temperature'].mean(),
        patient_vitals_monthly['temperature'].mean()],  # Use .mean() on 'temperature' column for each dataset
     'std_temperature': [patient_vitals_daily_agg['temperature'].std(),
        patient_vitals_weekly['temperature'].std(),
        patient_vitals_monthly['temperature'].std()]   # Use .std() on 'temperature' column for each dataset
 })

# TODO: Save results as 'output/q2_resampling_analysis.csv'
resampling_comparison.to_csv('output/q2_resampling_analysis.csv', index=False)

ICU daily shape: (180, 5)
Weekly resampled shape: (53, 5)
Monthly resampled shape: (12, 5)
Missing values after upsampling: temperature                 323
heart_rate                  323
blood_pressure_systolic     323
blood_pressure_diastolic    323
weight                      323
dtype: int64


## Part 2.4: Missing Data Handling

**💡 TIP: High Percentage of Missing Data is Expected!**
When upsampling from monthly to daily frequency, you'll create approximately 96% missing data (only 12 month-end dates have values out of 365 days). This is normal and expected for upsampling - don't be alarmed!

**Approach:** Create missing values by upsampling monthly data to daily frequency. This creates a clear, structured pattern of missing data that's ideal for practicing imputation methods.

**TODO: Handle missing data in time series**

In [38]:
# TODO: Identify missing values in time series
# Use the monthly resampled data from Part 2.3 and upsample to daily:
#   - Take patient_vitals_monthly['temperature']
#   - Upsample to daily frequency using .resample('D').asfreq()
#   - This creates missing values for all days except month-end dates (~96% missing)
ts_with_missing = patient_vitals_monthly['temperature'].resample('D').asfreq()  # Time series with missing values
print("Missing value count:", ts_with_missing.isna().sum())
print("Missing value percentage:", ts_with_missing.isna().sum() / len(ts_with_missing) * 100)

# TODO: Use forward fill and backward fill
ts_ffill = ts_with_missing.ffill()  # Forward fill missing values (use .ffill() method)
ts_bfill = ts_with_missing.bfill()  # Backward fill missing values (use .bfill() method)

# TODO: Use interpolation methods
ts_interpolated = ts_with_missing.interpolate()  # Interpolate missing values
ts_interpolated_linear = ts_with_missing.interpolate(method='linear')  # Linear interpolation
ts_interpolated_time = ts_with_missing.interpolate(method='time')  # Time-based interpolation
print("Missing value count after time-based interpolation:", ts_interpolated_time.isna().sum())
print("Missing value percentage after time-based interpolation:", ts_interpolated_time.isna().sum()/len(ts_interpolated_time) *100)

# TODO: Use rolling mean for imputation
ts_roll = ts_ffill.rolling(window=5, min_periods = 1).mean() # Have to use ffill data because this takes care of the trailing missing values, just using rolling mean doesn't work
ts_rolling_imputed = ts_ffill.where(ts_ffill.notna(), ts_roll)
print("Missing value count after rolling mean imputation:", ts_rolling_imputed.isna().sum())
print("Missing value percentage afetr rolling mean imputation:", ts_rolling_imputed.isna().sum()/len(ts_rolling_imputed) *100)
 # Fill missing with rolling mean

print(ts_with_missing)
print(ts_interpolated_time)

# TODO: Create missing data report
# Document your missing data handling with the following sections:
# 1. Missing value summary: Total count and percentage
# 2. Missing data patterns: When/why data is missing (by month, day of week, etc.)
# 3. Imputation method: Which method you used (forward fill, backward fill, interpolation, rolling mean)
# 4. Rationale: Why you chose that method
# 5. Pros and cons: Advantages and limitations of your approach
# 6. Example: Show at least one example of missing data before and after imputation
# Minimum length: 300 words
missing_data_report = """TODO: Document your missing data handling:
- Missing value summary: There were 323 missing values found, about 96.42 percent of the data was missing.
This was among multiple columns, most missing values were found in heart rate, blood pressure, and weight.
- Missing data patterns: We used multiple imputation methods including forward fill, backward fill,
     interpolation, linear interpolation, time-base interpolation and rolling mean imputation. My preference was towards the time-base interpolation based on the percentage of the data missing,
     and my understanding of interpolation using time as a third variable to help with imputation of missing values. Another method I was leaning towards using was a more basic forward fill, 
     however, because there is so much data missing, I am hesitant to use this method as it may cause overestimation of values that are not actually reflective of the true data.
- Rationale: I believe this to be a more reliable way to impute data.
     Using the wrong imputation method can lead to negative outcomes when more data is missing such as extreme skews or incorrect representations of the overall trends.
- Pros and Cons: Some pros with time-base interpolation is that the data will not stray too far from the original data points and we can use this in a larger dataset as a way 
to keep consistency with trends that may be observed. However, the same risks apply as with any form of imputation where if you overfit a dataset to a trend you observe, i.e. you 
impute too many datasets, you can lead to something disastrous like creating your own data based off of minimal data points that are collected. So, in general, it is always safer to 
be more conservative with imputation methods no matter how precise they may be. You can also use more than one method to keep variation and avoid some form of overfitting or over-imputation 
if you will.
- Example:
date
2023-01-31    98.777000
2023-02-01          NaN
2023-02-02          NaN
2023-02-03          NaN
2023-02-04          NaN
                ...    
2023-12-27          NaN
2023-12-28          NaN
2023-12-29          NaN
2023-12-30          NaN
2023-12-31    98.538865
Freq: D, Name: temperature, Length: 335, dtype: float64
date
2023-01-31    98.777000
2023-02-01    98.785011
2023-02-02    98.793021
2023-02-03    98.801031
2023-02-04    98.809041
...
2023-12-29    98.523725
2023-12-30    98.531295
2023-12-31    98.538865

"""

# TODO: Document missing data patterns
# Analyze when/why data is missing
missing_by_month = ts_with_missing.groupby(ts_with_missing.index.month).apply(lambda x: x.isna().sum())
missing_by_day = ts_with_missing.groupby(ts_with_missing.index.dayofweek).apply(lambda x: x.isna().sum())
missing_patterns = f"Missing by month:\n{missing_by_month}\n\nMissing by day of week:\n{missing_by_day}"

# TODO: Save results as 'output/q2_missing_data_report.txt'
with open('output/q2_missing_data_report.txt', 'w') as f:
     f.write(missing_data_report)
     f.write(f"\n\nMissing patterns:\n{missing_patterns}")

Missing value count: 323
Missing value percentage: 96.41791044776119
Missing value count after time-based interpolation: 0
Missing value percentage after time-based interpolation: 0.0
Missing value count after rolling mean imputation: 0
Missing value percentage afetr rolling mean imputation: 0.0
date
2023-01-31    98.777000
2023-02-01          NaN
2023-02-02          NaN
2023-02-03          NaN
2023-02-04          NaN
                ...    
2023-12-27          NaN
2023-12-28          NaN
2023-12-29          NaN
2023-12-30          NaN
2023-12-31    98.538865
Freq: D, Name: temperature, Length: 335, dtype: float64
date
2023-01-31    98.777000
2023-02-01    98.785011
2023-02-02    98.793021
2023-02-03    98.801031
2023-02-04    98.809041
                ...    
2023-12-27    98.508585
2023-12-28    98.516155
2023-12-29    98.523725
2023-12-30    98.531295
2023-12-31    98.538865
Freq: D, Name: temperature, Length: 335, dtype: float64


## Submission Checklist

Before moving to Question 3, verify you've created:

- [ ] `output/q2_resampling_analysis.csv` - resampling analysis results
- [ ] `output/q2_missing_data_report.txt` - missing data handling report
